In [30]:
import pandas as pd
from pandas.api import types as ptypes
import numpy as np
import chardet

import os 
from scipy import stats

In [31]:
import os

cwd = os.getcwd()

print(cwd)

c:\Users\Joel David\OneDrive\Desktop\Rut\Rut UNAM\2do semestre\Bioestadística\Statacode


In [32]:
from pathlib import Path

cwd = Path.cwd()

print("Current directory:", cwd)

Current directory: c:\Users\Joel David\OneDrive\Desktop\Rut\Rut UNAM\2do semestre\Bioestadística\Statacode


In [33]:
data_idir='Data/'
filename='data_basal_musculoesqueleticos.csv'

In [34]:
raw=open(data_idir+filename,'rb').read(10000)
result=chardet.detect(raw)
encoding=result['encoding']
print(f"Detected encoding:{encoding}")
df=pd.read_csv(data_idir+filename,encoding=encoding)  

Detected encoding:utf-8


In [35]:
def infer_column_type(series, cat_threshold=0.05,   unique_thresh =10):
    if ptypes.is_categorical_dtype(series) or ptypes.is_object_dtype(series):
        return 'categorical'
    
    if ptypes.is_numeric_dtype(series):
             n=len(series)
             nunq=series.nunique(dropna=True)
             frac=nunq/n

             if nunq<= unique_thresh or frac <=  cat_threshold:
                 return 'categorical'
             else: 
                 return 'numeric'
    return 'unknown'

def  cast_column_by_type(df, cat_threshold=0.05, unique_threshold=15):
    for col in df.columns:
        col_type=infer_column_type(df[col],cat_threshold, unique_threshold)
        if col_type=='categorical':
           df[col] = df[col].astype('category')
        elif col_type=='numeric':
            df[col]=pd.to_numeric(df[col],errors='coerce')
    return df

In [28]:
df_type=cast_column_by_type(df)

C:\Users\Joel David\AppData\Local\Temp\ipykernel_15480\861121882.py:2: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if ptypes.is_categorical_dtype(series) or ptypes.is_object_dtype(series):
C:\Users\Joel David\AppData\Local\Temp\ipykernel_15480\861121882.py:2: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if ptypes.is_categorical_dtype(series) or ptypes.is_object_dtype(series):
C:\Users\Joel David\AppData\Local\Temp\ipykernel_15480\861121882.py:2: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if ptypes.is_categorical_dtype(series) or ptypes.is_object_dtype(series):
C:\Users\Joel David\AppData\Local\Temp\ipykernel_15480\861121882.py:2: DeprecationWarning: is_categorical_dtype is dep

In [57]:
def summarize_and_test(df,group_col,alpha=0.05):
    numeric_cols=df.select_dtypes(include=[np.number]).columns.drop(group_col, errors='ignore')
    cat_cols=df.select_dtypes(include=['category','object','bool'] ).columns.drop(group_col,errors='ignore')
    groups=df[group_col].dropna().unique()
    results=[]

    #format p-value
    fmt_p = lambda p: f"{p:.3g}"

    for var in numeric_cols:
       data_by_g=[df.loc[df[group_col]==g,var].dropna() for g in groups]
       #normality tst
       sw_p=[stats.shapiro(x)[1] if len(x)>=3 and len(x)<=5000 else 1.0 for x in data_by_g]
       
       all_normal =  all([p>alpha for p in sw_p])
       if len(groups)== 2:
           g1,g2=data_by_g
           if all_normal:
               test_name,test_stat, p='t-test',*stats.ttest_ind(g1,g2,nan_policy='omit')
           else:
               test_name,test_stat,p='Mann-Whitney U',*stats.mannwhitneyu(g1,g2,alternative='two-sided')
       else:
               if all_normal:
                   test_name,test_stat, p='ANOVA',*stats.f_oneway(*data_by_g)
               else:
                   test_name,test_stat,p='Krustal-Wallis',*stats.kruskal(*data_by_g)

       Sumaries={
           g: f"{df.loc[df[group_col]==g,var].mean():.2f} ({df.loc[df[group_col]==g,var].std(ddof=1):.2f})"
           for g in groups
                  }  
           
       results.append({
           'Variable': var,
           'Test':test_name,
           'Statistic': test_stat,
           'p_value':fmt_p(p),
           **Sumaries
       })

       #categorical variables
       for var in cat_cols:
           ct=pd.crosstab(df[var],df[group_col])
           #expectde counts
           chi2,chi2_p,dof, exp=stats.chi2_contingency(ct,correction=False)
           #check if all expected counts are greather than 5
           use_fisher=(ct.shape==(2,2) and (exp<5).any())
           if use_fisher:
               #fisher´s exact test
               _, fisher_p=stats.fisher_exact(ct)
               test_name='Fisher\'s Exact Test'
               p=chi2_p
           else:
               #chi-square test
               test_name='Chi-squared Test'
               p=chi2_p
               
           summaries ={}
           for g in groups:
               col_tot= ct[g].sum()
               summaries[g]=';'.join( f"{level}:{ct.loc[level,g]} ({ct.loc[level,g]/col_tot*100:.1f}%)"
               for level in ct.index
            )
               results.append({
                   'Variable':var,
                   'Test':test_name,
                   'Statistic': None,
                   'p-value':fmt_p(p),
                   **summaries
               })

    return pd.DataFrame(results)

In [58]:
df_test=df_type.copy()
df_test=df_test.drop(columns=['PatientId'])
df_test['Diagnostico_bin']=np.where(df_test['Diagnostico']!='Ninguno',1,0)
df_test=df_test.drop(columns=['Diagnostico'])
summary_table=summarize_and_test(df_test, group_col='Diagnostico_bin')

In [59]:
print(summary_table.to_markdown(index=False))

ImportError: Missing optional dependency 'tabulate'.  Use pip or conda to install tabulate.

In [ ]:
#para cambiar por os
# 

data_folder = Path("Data")  working directory

file_path = data_folder / "example.txt"

data_folder = Path("Data") # relative to your current 

file_path = data_folder / "example.txt"

out_folder = Path("out")

out_folder.mkdir(parents=True, exist_ok=True) 

out_file = out_folder / "output.txt"